In [ ]:
#code with autotune


import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
import os
#XLA Activating
tf.config.optimizer.set_jit(True) # وقتی منسبه که سایز ورودی همه بچ ها ثابت باشه..مثلا همش 224*224 باشه ولی در غیر این صورت نه!

# Define the train and validation folder paths
train_folder = 'C:/Users/salir/OneDrive/Desktop/catvsdogs/train'
val_folder = 'C:/Users/salir/OneDrive/Desktop/catvsdogs/train'

# Function to preprocess images with augmentation
def preprocess_image_aug_norm(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
  # Data augmentation
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
  # Normalize using VGG16 preprocessing
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image, label

# Function to preprocess images without augmentation

#We use tf.function primarily on functions that involve TensorFlow operations. These are the instructions that involve lots of calculations, like processing images or making predictions with a neural network.

#When we add @tf.function before a function definition, we're telling TensorFlow to convert that function into a special format called a graph. This graph is optimized for faster execution, especially on GPUs or TPUs.

#For other calculations that don't involve TensorFlow operations, like simple arithmetic or basic Python functions, we don't need to use tf.function. That's because these calculations are already pretty fast and efficient in regular Python.

#So, in summary, we use tf.function on functions that involve TensorFlow operations to speed up those specific parts of our code, while leaving other calculations as they are since they're already efficient. It's like using a turbo boost for the parts of our code that need it most!

#tf.function Activation
@tf.function
def preprocess_image_norm(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image, label

# Load and preprocess the training dataset with augmentation

#num_parrarel_calls Actiavtion to activate treds of CPU
#prefetch Activation to import next batch pararelly while training

train_dataset = tf.data.Dataset.list_files(train_folder + '/*/*')
train_dataset = train_dataset.map(lambda x: (x, tf.strings.split(x, os.path.sep)[-2] == 'cat'),num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.map(lambda x, y: (preprocess_image_aug_norm(x, y)), num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(128)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Load and preprocess the validation dataset without augmentation
val_dataset = tf.data.Dataset.list_files(val_folder + '/*/*')
#
val_dataset = val_dataset.map(lambda x: (x, tf.strings.split(x, os.path.sep)[-2] == 'cat'),num_parallel_calls=tf.data.experimental.AUTOTUNE)
#or
#def get_label(filepath):
  #subdirectory = tf.strings.split(filepath, os.path.sep)[-2]
  #return filepath, subdirectory == 'cat'
  #train_dataset = train_dataset.map(get_label)
   
val_dataset = val_dataset.map(lambda x, y: (preprocess_image_norm(x, y)), num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(128)
val_dataset = val_dataset.cache()# به شرطی که دیتا نوی رم جا بشه و خیلی وریشن نداشته باشه.برای دیتای ترین و آوگمنتیشن خیلی مناسب نیست
#بعد از اینکه دیتای ولیدیشن رو کش کردیم و قبل از اینکه پری فچ کنیم، دیتا رو کش میکنیم
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE) #prefetch next batch while training

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of VGG16
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with a specified learning rate
learning_rate = 0.1
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=1)
